In [2]:
pip install tensorflow

     |████████████████████████████████| 462 kB 2.8 MB/s 


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix


In [4]:
def norm(x, lim_sup=1, lim_inf=-1):
  return lim_inf + (x - x.min(axis=0))*(lim_sup-lim_inf) / (x.max(axis=0) - x.min(axis=0))

In [5]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [6]:
class MLP(object):

    def __init__(self, optimizer, *args, **kwargs):
        
        self.shape = args
        self.neuron = len(args)
        options = {
            'hidden_activation' : 'relu',
            'out_activation'    : 'softmax'
            }

        options.update(kwargs)
        self.inputDim = args[0]
        self.hidden_activation = options['hidden_activation']
        self.out_activation= options['out_activation']
        self.optimizer = optimizer

        self.build_model()
    
    def build_model(self):
      self.model = Sequential()
      # self.model.add(Dense(10, input_shape=(4,), activation='relu', name='fc1'))
      # self.model.add(Dense(10, activation='relu', name='fc2'))
      # self.model.add(Dense(3, activation='softmax', name='output'))
      for i in range(1, self.neuron):

        name = str(i)
        if i == 1:
          self.model.add(Dense(self.shape[i], name=name, input_shape=(self.shape[0],), activation = self.hidden_activation))
						# contruindo a camada de entrada, juntamente com os neuronios da camada escondida
        elif i == (self.neuron-1):
            self.model.add(Dense(self.shape[i], name='out_layer',activation = self.out_activation))
						# construindo a camada de saida
        else:
            self.model.add(Dense(self.shape[i], name=name, activation = self.hidden_activation))
            # contruindo camada escondidas. caso o modelo contenha mais de uma camada escondida 
                                              
      self.model.compile(optimizer=self.optimizer, loss='mean_squared_error', metrics=['accuracy']) 

      print('Neural Network Model Summary: ')
      print(self.model.summary())

    def train_model(self, trainX, trainY, valX=None, valY=None, epoch=100, batchSize=5, verbose=1):
      
      if valX==None or valY==None:
        validation_data=None
      else:
        validation_data = (valX, valY)

      self.history = self.model.fit(trainX, trainY, epochs=epoch, validation_data = validation_data, 
                                    validation_split=0.2, batch_size=batchSize, verbose=verbose)
      
    def test_model(self, testX, testY):
      
      self.score = self.model.evaluate(testX, testY, verbose=1)

# Iris

In [7]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

iris_data = load_iris() # load the iris dataset

x = iris_data.data

x = norm(x)
y = iris_data.target.reshape(-1, 1) # Convert data to a single column

# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Split the data for training and testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)

print('Example data: ')
print(train_x[:5])
print('\nExample labels: ')
print(train_y[:5])

Example data: 
[[ 1.          0.5         0.83050847  0.58333333]
 [ 0.22222222 -0.16666667  0.42372881  0.58333333]
 [-0.72222222 -0.16666667 -0.86440678 -1.        ]
 [-0.83333333 -0.08333333 -0.83050847 -0.91666667]
 [ 0.44444444 -0.08333333  0.38983051  0.83333333]]

Example labels: 
[[0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [8]:
# creating an classifier from the model:
optimizer = SGD(learning_rate=0.01, momentum=0.9)
# optimizer =  Adam(lr=0.001)

mlp = MLP(optimizer, 4, 10, 10, 3)
mlp.train_model( train_x,  train_y, epoch=200, batchSize=5, verbose=0)
mlp.test_model(test_x, test_y)

Neural Network Model Summary: 
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 1 (Dense)                   (None, 10)                50        
                                                                 
 2 (Dense)                   (None, 10)                110       
                                                                 
 out_layer (Dense)           (None, 3)                 33        
                                                                 
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 124ms/step - loss: 0.0321 - accuracy: 0.9000
